In [1]:
using Rx
using BenchmarkTools

┌ Info: Precompiling Rx [df971d30-c9d6-4b37-b8ff-e965b2cb3a40]
└ @ Base loading.jl:1273


In [2]:
function withRegularArray()
    values = [ i for i in 1:100 ]
    keep    = Vector{Vector{Int}}()
    current = Int[]
    for value in values
        current = [ current..., value ]
        push!(keep, current)
    end
    return keep
end

withRegularArray (generic function with 1 method)

In [3]:
struct KeepActor{D} <: NextActor{D}
    values::Array{D, 1}
    
    KeepActor{D}() where D = new(Array{D, 1}())
end

Rx.on_next!(actor::KeepActor{D}, data::D) where D = push!(actor.values, data)

In [4]:
@CreateScanOperator(IntoArray, (d, c) -> [ c..., d ])

function withPregeneratedOperator()
    source = from([ i for i in 1:100 ])
    keep = KeepActor{Array{Int, 1}}()
    subscribe!(source |> IntoArrayScanOperator{Int, Array{Int, 1}}(Int[]), keep)
    return keep.values
end

withPregeneratedOperator (generic function with 1 method)

In [5]:
function withBaseScanOperator()
    source = from([ i for i in 1:100 ])
    keep = KeepActor{Array{Int, 1}}()
    subscribe!(source |> scan(Int, Vector{Int}, (d, c) -> [ c..., d ], Int[]), keep)
    return keep.values
end

withBaseScanOperator (generic function with 1 method)

In [6]:
@btime withRegularArray()

  65.426 μs (209 allocations: 53.78 KiB)


100-element Array{Array{Int64,1},1}:
 [1]                                                                        
 [1, 2]                                                                     
 [1, 2, 3]                                                                  
 [1, 2, 3, 4]                                                               
 [1, 2, 3, 4, 5]                                                            
 [1, 2, 3, 4, 5, 6]                                                         
 [1, 2, 3, 4, 5, 6, 7]                                                      
 [1, 2, 3, 4, 5, 6, 7, 8]                                                   
 [1, 2, 3, 4, 5, 6, 7, 8, 9]                                                
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]                                            
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]                                        
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]                                    
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12

In [7]:
@btime withPregeneratedOperator()

  69.961 μs (215 allocations: 53.97 KiB)


100-element Array{Array{Int64,1},1}:
 [1]                                                                        
 [1, 2]                                                                     
 [1, 2, 3]                                                                  
 [1, 2, 3, 4]                                                               
 [1, 2, 3, 4, 5]                                                            
 [1, 2, 3, 4, 5, 6]                                                         
 [1, 2, 3, 4, 5, 6, 7]                                                      
 [1, 2, 3, 4, 5, 6, 7, 8]                                                   
 [1, 2, 3, 4, 5, 6, 7, 8, 9]                                                
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]                                            
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]                                        
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]                                    
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12

In [8]:
@btime withBaseScanOperator()

  82.172 μs (315 allocations: 57.11 KiB)


100-element Array{Array{Int64,1},1}:
 [1]                                                                        
 [1, 2]                                                                     
 [1, 2, 3]                                                                  
 [1, 2, 3, 4]                                                               
 [1, 2, 3, 4, 5]                                                            
 [1, 2, 3, 4, 5, 6]                                                         
 [1, 2, 3, 4, 5, 6, 7]                                                      
 [1, 2, 3, 4, 5, 6, 7, 8]                                                   
 [1, 2, 3, 4, 5, 6, 7, 8, 9]                                                
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10]                                            
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11]                                        
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12]                                    
 [1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12